In [1]:
import numpy as np 
import pandas as pd

In [2]:
pr = pd.read_csv('diabetes.csv')
pr.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# corelation of output with other parameters 
# we can remove which are not important parameters

pr.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
# in x we provide input data and in y we put output data
x = pr.iloc[:,:-1].values
y = pr.iloc[:,-1].values

In [5]:
#scaling the data (0-1)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [6]:
x.shape

(768, 8)

In [7]:
# to train the data 
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.2,random_state=1)

In [8]:
#to create a model
import tensorflow
from tensorflow import keras
from tensorflow.keras import  Sequential
from tensorflow.keras.layers import  Dense

In [9]:
# create node and layer in model
model = Sequential()

model.add(Dense(32,activation='relu',input_dim = 8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer= 'Adam',loss='binary_crossentropy',metrics=['accuracy'])


C:\Users\dt160\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
#generate a model according to parameters

model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.6249 - loss: 0.7496 - val_accuracy: 0.6309 - val_loss: 0.7144
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6263 - loss: 0.7331 - val_accuracy: 0.6407 - val_loss: 0.6983
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6189 - loss: 0.7159 - val_accuracy: 0.6520 - val_loss: 0.6832
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6510 - loss: 0.7080 - val_accuracy: 0.6585 - val_loss: 0.6697
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7001 - loss: 0.6693 - val_accuracy: 0.6634 - val_loss: 0.6571
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6710 - loss: 0.6589 - val_accuracy: 0.6650 - val_loss: 0.6452
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6762 - loss: 0.6463 - val_accuracy: 0.6683 - val_loss: 0.6344
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6384 - loss: 0.6461 - val_accuracy: 0.6699 - val_loss: 0.6243


In [11]:
# to automate the whole process of creating node and layer by hypertuning
#pip install keras-tuner
import kerastuner as kt

C:\Users\dt160\AppData\Local\Temp\ipykernel_12800\1956488146.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [12]:
#creating a model for using the hypertuner
def build_model(hp):
    model = Sequential()

    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])

    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [13]:
# tuner for model
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [14]:
# it search the best tuning setting for the best result with this model
# the weights are save in new creted folder
tuner.search(x_train,y_train,epochs=5,validation_data = (x_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6861788630485535

Best val_accuracy So Far: 0.6861788630485535
Total elapsed time: 00h 00m 10s


In [15]:
# it shows which optimizer is better for this model
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [16]:
# this code search the best model (high accuracy)
model = tuner.get_best_models(num_models=1)[0]

C:\Users\dt160\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=7,validation_data=(x_test,y_test))

Epoch 8/100


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.6563 - loss: 0.6266 - val_accuracy: 0.6878 - val_loss: 0.6095
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7053 - loss: 0.5935 - val_accuracy: 0.6943 - val_loss: 0.6001
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7032 - loss: 0.5749 - val_accuracy: 0.6992 - val_loss: 0.5919
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7010 - loss: 0.5946 - val_accuracy: 0.7008 - val_loss: 0.5848
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7279 - loss: 0.5614 - val_accuracy: 0.7089 - val_loss: 0.5779
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6737 - loss: 0.5881 - val_accuracy: 0.7106 - val_loss: 0.5720
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.7331 - loss: 0.5614 - val_accuracy: 0.7154 - val_loss: 0.5664
Epoch 15/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7036 - loss: 0.5648 - val_accuracy: 0.7203 - val_loss: 0.561

In [20]:
# right number of nurons for the model
#find the num of nurons right fot this model through hyperparameter

def build_model(hp):
    model = Sequential()

    units = hp.Int('units',8,128,step=8) 

    model.add(Dense(units,activation='relu',input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model



In [21]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5
                        )

Reloading Tuner from .\untitled_project\tuner0.json


In [22]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [23]:
# it shows the all trial improvement and important parameters
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
optimizer: adam
Score: 0.6861788630485535

Trial 2 summary
Hyperparameters:
optimizer: sgd
Score: 0.6552845239639282

Trial 0 summary
Hyperparameters:
optimizer: adadelta
Score: 0.6195122003555298

Trial 1 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.5056910514831543


In [24]:
# to see the best hyperparamer
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [ ]:
# to see the best model from the trials
model = tuner.get_best_models(num_models=1)[0]


ValueError: A total of 2 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(8, 8), Received: value.shape=(8, 32). Target variable: <Variable path=sequential/dense/kernel, shape=(8, 8), dtype=float32, value=[[ 0.02893537  0.24388164  0.22953951  0.10918272  0.25591213  0.05029619
   0.5534242   0.01508147]
 [-0.09410012 -0.33187306  0.30035025  0.27237856  0.34034932 -0.30670744
  -0.05060905  0.5963935 ]
 [ 0.60851365  0.17513078 -0.20549154  0.4617694   0.39482135  0.5244494
   0.48088926  0.50320894]
 [ 0.01222831  0.47904032  0.05915743  0.03298587  0.59177405  0.49690384
   0.22610176 -0.24820733]
 [-0.16771585 -0.33569637 -0.28962988  0.51302856 -0.58718336  0.5478125
   0.3524164   0.49543256]
 [ 0.33273667 -0.5349643   0.21176201 -0.41488934 -0.04463059  0.12827688
   0.48456627 -0.32644853]
 [-0.2040019   0.405226

In [26]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8188 - loss: 0.3710 
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7888 - loss: 0.3929 
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7918 - loss: 0.4112 
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8400 - loss: 0.3541
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8231 - loss: 0.3620 
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8066 - loss: 0.4145 
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8005 - loss: 0.3881 
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8192 - loss: 0.3566 
Epoch 15/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7819 - loss: 0.4039 
Epoch 16/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8274 - loss: 0.3646 
Epoch 17/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8096 - loss: 0.3723
Epoch 18/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7788 - loss: 0.

In [27]:
def build_model(hp):

    model = Sequential()

    model.add(Dense(72,activation='relu',input_dim=8))

    for i in range(hp.Int('num_layers',1,10)):
        model.add(Dense(72,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))    

    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

    

In [28]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='num_layer')

C:\Users\dt160\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7430894374847412

Best val_accuracy So Far: 0.7642276287078857
Total elapsed time: 00h 00m 10s


In [30]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [31]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100


C:\Users\dt160\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.7970 - loss: 0.5155 - val_accuracy: 0.7691 - val_loss: 0.5249
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7918 - loss: 0.4828 - val_accuracy: 0.7659 - val_loss: 0.5099
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.7862 - loss: 0.4664 - val_accuracy: 0.7642 - val_loss: 0.5003
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8170 - loss: 0.4197 - val_accuracy: 0.7659 - val_loss: 0.4971
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7562 - loss: 0.4551 - val_accuracy: 0.7707 - val_loss: 0.4990
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8075 - loss: 0.3999 - val_accuracy: 0.7691 - val_loss: 0.5022
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7944 - loss: 0.4103 - val_accuracy: 0.7626 - val_loss: 0.5076
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8005 - loss: 0.3821 - val_accuracy: 0.7626 - val_loss: 0.5098